In [266]:
import pandas as pd
import numpy as np
import fnmatch
import matplotlib
import matplotlib.pyplot as plt
import unidecode
import re

from os import listdir
from os.path import isfile, join

In [17]:
datasets_path = "../data/raw/accidents/"

dataset_paths = [file for file in listdir(datasets_path)]

print(dataset_paths)

['2016_accidents.csv']


# Step 1: Load dataframes data into list of DFs

In [103]:
def extract_accident_name(filename):
    return filename.split("_")[0]

def init_df(file):
    name = extract_accident_name(file)
    df = pd.read_csv(datasets_path+file)
    df["name"] = name
    return df

accidents_df_list = [init_df(file) for file in dataset_paths]

# Step 2: Find wrong/missing data
## Step 2.1: Normalize column names
    1) Swap space " " for underscore "_" and remove apostrophe "'"
    2) Deunicode and lowercase columns names

In [95]:
def clean_headers(df):
    df.columns = df.columns.str.replace(' ', '_').str.replace('\'', '')
    df.rename(columns=lambda c: unidecode.unidecode(c.lower()), inplace=True)
    return df

accidents_df = [clean_headers(df) for df in accidents_df_list]
accidents_df[0].head(1)

,numero_dexpedient,codi_districte,nom_districte,codi_barri,nom_barri,codi_carrer,nom_carrer,num_postal_caption,descripcio_dia_setmana,dia_setmana,...,descripcio_sexe,edat,descripcio_tipus_persona,descripcio_situacio,descripcio_victimitzacio,coordenada_utm_(x),coordenada_utm_(y),long,lat,name
0,2016S008630,5,Sarrià-Sant Gervasi,22,"Vallvidrera, el Tibidabo i les Planes",202050,Mas Guimbau,0001B0001B,Dijous,Dj,...,Home,29,Conductor,Presentado,Ferit lleu,"424249,09","4586527,04",2.092323,41.424889,2016


## Step 2.2: Find inconsistent data among Dataframes rows.

In [278]:
def clean_col(df, col):
    df[col] = df[col].dropna()
    return df

def delete_multispaces(row):
    for index, value in row.items():
        if type(value) == str:
            value = value.strip()
            value = re.sub(r'(?<=[A-Za-z])( +)(?=[A-Za-z])', ' ', value)
            row[index] = re.sub(r'(?<=\d)( +)(?=\d)?', '', value)
    return row

cols_name = accidents_df[0].columns
col_size = len(accidents_df[0].columns)

for df in accidents_df:
    null_cols = list(df.columns[df.isna().any()])

    for null_col in null_cols:
        df = df[(df[null_col].notna()) & (df[null_col].replace(" ", ""))]

    del df["num_postal_caption"]

    df = df.apply(delete_multispaces)
    
    df.to_csv("out.csv", encoding='utf-8')